Name: Haw Xiao Ying <br>Student ID: 29797918 <br>Email: xhaw0001@student.monash.edu

# Assignment Part B

## Task 1(a)

In [ ]:
from pymongo import MongoClient
from pprint import pprint
import pandas as pd


client = MongoClient()
db = client.fit3182_assignment_db
climate = db.climate_historic

cursor = db.climate.find().sort("date",-1).limit(1)

date = pd.to_datetime(cursor[0]['date'])

print("The latest date in climate historic is", cursor[0]['date'], ", according to data below:")
for i in cursor:
    pprint(i)

In [ ]:
from time import sleep
from json import dumps
from kafka import KafkaProducer
import random, csv
import datetime as dt

 
def publish_message(producer_instance, topic_name, data):
    try:
        key_bytes = bytes("producer_1", encoding='utf-8')
        value_bytes = bytes(data, encoding='utf-8')
        producer_instance.send(topic_name, value=value_bytes, key=key_bytes)
        print('Message published successfully. ' + data)
    except Exception as ex:
        print('Exception in publishing message.')
        print(str(ex))
        

        
def connect_kafka_producer():
    _producer = None
    try:
        _producer = KafkaProducer(bootstrap_servers=['127.0.0.1:9092'], api_version=(0, 10))
    except Exception as ex:
        print('Exception while connecting Kafka.')
        print(str(ex))
    finally:
        return _producer
    

    
if __name__ == '__main__':
   
    topic = 'assignment'
    print('Publishing records..')
    producer = connect_kafka_producer()
            
    list_of_climate = []
    
    climate_streaming_filepath = '/home/student/Downloads/Assignment/climate_streaming.csv'
    with open(climate_streaming_filepath) as file:
        climate_csv = csv.DictReader(file)
        for each_row in climate_csv:
            list_of_climate.append(each_row)
        
        
    while True:
        random_number = random.randint(0,len(list_of_climate))
        each_row = list_of_climate[random_number]

        date += dt.timedelta(days=1)

        each_row["date"] = str(date.strftime('%Y-%m-%d'))
        each_row["station"] = 948703
        each_row["windspeed_knots"] = float(each_row["windspeed_knots"])
        each_row["max_wind_speed"] = float(each_row["max_wind_speed"])
        each_row["latitude"] = float(each_row["latitude"])
        each_row["longitude"] = float(each_row["longitude"])
        each_row["relative_humidity"] = float(each_row["relative_humidity"])
        each_row["air_temperature_celcius"] = int(each_row["air_temperature_celcius"])
        each_row["GHI_w/m2"] = int(each_row["GHI_w/m2"])
        each_row["producer"] = "producer_1"

        data = str(dumps(each_row))
        publish_message(producer, topic, data)
        sleep(10)